In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import whichBisSig_NAHS, B_ID, ancestor_below_B

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-26 09:48:11.966634


In [130]:
save_data = True
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [15]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "NAHSA_Gmodes_1/"
root_subdir = "NAHS_allEvts_oneSub/"  

root_path = nfs_path + "NAHS/onlineDataProd/" + root_subdir 
#root_path = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/"

In [5]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"
merged = ""

In [6]:
fileHc = uproot.open(root_path + merged + "allHc.root:variables")

In [7]:
df_Hc = fileHc.arrays(library="pd")

In [8]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename=root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/muons.root:variables


In [9]:
df_FSPs = pd.concat(dfs)

In [10]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [11]:
df_FSPs.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'cmpx', 'cmpy', 'cmpz', 'cmE', 'isSignal', 'uniqueParticleIdentifier',
       'mcErrors', 'mcPDG', 'genMotherID', 'genMotherP', 'genMotherPDG',
       'charge', 'dr', 'dz', 'clusterReg', 'clusterE9E21', 'M', 'PDG',
       'genParticleID', 'px', 'py', 'pz', 'pt', 'p', 'E'

In [12]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print(df_Hc["__event__"].nunique())

27043483
1622099
1532609


## check that each particles occurs only ones; count must be < 1

In [14]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID.value_counts('count')

count
1    27043483
dtype: int64

## add cols

In [16]:
df_Hc['Bsig_uniqParID'] = df_Hc.apply(whichBisSig_NAHS, axis=1)

In [17]:
df_Hc['Bsig_uniqParID'].value_counts()

83886081.0    852568
83886082.0    769531
Name: Bsig_uniqParID, dtype: int64

In [18]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [19]:
df_FSPs['B_ID'].value_counts()

83886081    11717640
83886082    11716799
0            3609044
Name: B_ID, dtype: int64

In [20]:
df_FSPs['PDG'].value_counts()

 22.0     14235671
 211.0     3350074
-211.0     3172959
 13.0      1381029
-13.0      1351792
 321.0      962201
-321.0      952602
-11.0       826266
 11.0       810889
Name: PDG, dtype: int64

In [21]:
df_Hc['PDG'].value_counts()

 413.0     525558
-413.0     512128
 411.0     197265
-411.0     197056
 421.0      75177
-421.0      49194
-431.0      17604
 431.0      17475
 423.0      10059
 443.0       7897
-423.0       6648
-4122.0      3074
 4122.0      2964
Name: PDG, dtype: int64

## check D*lnu

In [22]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886082.0].groupby( 
    ["genUp4S_PDG_1_0","genUp4S_PDG_1_1","genUp4S_PDG_1_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_1_0,genUp4S_PDG_1_1,genUp4S_PDG_1_2,count
0,-413.0,-13.0,14.0,190642
1,-413.0,-11.0,12.0,191440
2,413.0,11.0,-12.0,192579
3,413.0,13.0,-14.0,194870


In [23]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886081.0].groupby( 
    ["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_0_0,genUp4S_PDG_0_1,genUp4S_PDG_0_2,count
0,-413.0,-13.0,14.0,210918
1,-413.0,-11.0,12.0,212804
2,413.0,11.0,-12.0,215045
3,413.0,13.0,-14.0,213801


## delete evts where Hc mother and Bsig overlap

In [25]:
print(df_Hc.shape[0])

1622099


In [27]:
# these events have Hc as a Bsig daughter, which is of course wrong
df_Hc[df_Hc['Bsig_uniqParID'] == df_Hc['mcMother0_uniqParID']].shape[0]

763136

In [28]:
df_Hc = df_Hc[df_Hc['Bsig_uniqParID'] != df_Hc['mcMother0_uniqParID']]

In [29]:
print(df_Hc.shape[0])

858963


In [30]:
print(df_FSPs.shape[0])
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]
print(df_FSPs.shape[0])

27043483
14943973


## take care of duplis in df_Hc, delete them, because with the prior, duplis here mean there must have been an error

In [32]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    828867
2     14569
3       285
4        23
5         1
6         1
dtype: int64

In [33]:
df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]>1]["__event__"])][["__event__",
    "uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID","px","E","x","genUp4S_PDG_1_0"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID,px,E,x,genUp4S_PDG_1_0
50,3679955,100663296.0,-421.0,511.0,83886081.0,83886082.0,-0.079056,1.939253,-0.042812,411.0
56,243575,100663296.0,-421.0,-511.0,83886081.0,83886082.0,0.325505,2.003441,-0.044740,-421.0
77,9744,100663296.0,-421.0,-511.0,83886081.0,83886082.0,0.841873,2.300964,-0.052184,411.0
94,14108,100663296.0,421.0,-511.0,83886081.0,83886082.0,-0.058585,1.946033,-0.046022,-321.0
95,14108,100663296.0,421.0,-511.0,83886081.0,83886082.0,0.619852,2.303867,-0.056158,-321.0
117,4558,100663296.0,421.0,-511.0,83886082.0,83886081.0,-0.296814,1.911936,-0.561030,-413.0
118,4558,100663296.0,421.0,-511.0,83886082.0,83886081.0,-0.300684,1.894096,-0.043345,-413.0
121,5600,100663296.0,-421.0,511.0,83886082.0,83886081.0,0.179031,2.151420,-0.047189,413.0
122,5600,100663296.0,421.0,511.0,83886082.0,83886081.0,-0.384867,2.601196,0.000000,413.0
124,5661,100663296.0,-421.0,-511.0,83886082.0,83886081.0,0.078100,1.885611,-0.044987,-413.0


In [34]:
df_Hc = df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]==1]["__event__"])]

In [35]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [36]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    828867
dtype: int64

## delete the H_c FSPs 

In [37]:
df_FSPs['ancestor_below_B_arrayID'] = df_FSPs.apply(ancestor_below_B, axis=1)

In [38]:
df_Hc["HcMother_uniqParID"] = df_Hc["mcMother0_uniqParID"]
df_Hc["Hc_genParticleID"] = df_Hc["genParticleID"]
df_Hc["Hc_MotherPDG"] = df_Hc["genMothPDG_0"]

In [39]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","Hc_MotherPDG"]], on="__event__")

In [40]:
df_FSPs[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID",
         "ancestor_below_B_arrayID","Hc_MotherPDG"]][:10]

,__event__,Bsig_uniqParID,HcMother_uniqParID,Hc_genParticleID,ancestor_below_B_arrayID,Hc_MotherPDG
0,2331192,83886082.0,83886081.0,3.0,0.0,511.0
1,2331192,83886082.0,83886081.0,3.0,0.0,511.0
2,2331192,83886082.0,83886081.0,3.0,6.0,511.0
3,2331192,83886082.0,83886081.0,3.0,4.0,511.0
4,2331192,83886082.0,83886081.0,3.0,6.0,511.0
5,2331192,83886082.0,83886081.0,3.0,6.0,511.0
6,2331192,83886082.0,83886081.0,3.0,0.0,511.0
7,2331192,83886082.0,83886081.0,3.0,4.0,511.0
8,2331192,83886082.0,83886081.0,3.0,6.0,511.0
9,2331192,83886082.0,83886081.0,3.0,3.0,511.0


In [44]:
df_FSPs.shape[0]

11163573

In [45]:
# deletion of Hc FSPs, those who 
df_FSPs = df_FSPs[~((df_FSPs["Hc_genParticleID"] == df_FSPs["ancestor_below_B_arrayID"]) &  # originate from Hc arrayID(genParticleID)
                  (df_FSPs["HcMother_uniqParID"] == df_FSPs["B_ID"]))] # and are B-tag ancestors

In [46]:
df_FSPs.shape[0]

11163573

## delete delete_top_numFSP_evts

In [47]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Hc.shape[0]

In [48]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Hc.shape[0]/df_Hc.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 35
0.995
would delete 0.8 % of evts
new max FSPs: 23
0.996
would delete 0.42 % of evts
new max FSPs: 24
0.997
would delete 0.42 % of evts
new max FSPs: 24
0.998
would delete 0.22 % of evts
new max FSPs: 25
0.999
would delete 0.11 % of evts
new max FSPs: 26


In [49]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.995
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print("deleted", round(100 - (df_Hc.shape[0]/numEvtsBeforeCut)*100,4),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())

11163573
828867
used delete_percentileLvl: 0.995
count    828867.000000
mean         13.468473
std           3.960402
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          35.000000
Name: count, dtype: float64
10996905
822214
deleted 9901.0 % of events
count    822214.000000
mean         13.374748
std           3.834275
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          23.000000
Name: count, dtype: float64


## impute the nan values of variables

In [50]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.],["protonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0
0


## last check if events match between dfs

In [51]:
df_Hc = df_Hc[df_Hc['__event__'].isin(df_FSPs["__event__"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [52]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

10996905
822214


## check the subset of (B-sig side) D*-> D0 (-> kpi, kpipi,...) pi  

In [53]:
df_Hc.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'extraInfo__boSignalProbability__bc',
       'isSignalAcceptMissingGamma', 'PDG', 'genUp4S_uniqParID_0',
       ...
       'z', 'x_uncertainty', 'y_uncertainty', 'z_uncertainty',
       'uniqueParticleIdentifier', 'genParticleID', 'Bsig_uniqParID',
       'HcMother_uniqParID', 'Hc_genParticleID', 'Hc_MotherPDG'],
      dtype='object', length=103)

In [109]:
# select only these D0 modes and D*+ -> D0 pi
#D0:kpi 
#D0:kpipi pi 
#D0:kpipi0     
#D0:kpipi pi pi0 

In [119]:
pd.options.display.float_format = '{:20.2f}'.format
df_Hc = df_Hc[((df_Hc["Bsig_uniqParID"] == 83886081.0)  &
            (df_Hc["genUp4S_PDG_0_0_0"].abs() == 421.0)  & # 421=D0
            (df_Hc["genUp4S_PDG_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Hc["genUp4S_PDG_0_0_2"].isna() == True)  & # no particle 
            (df_Hc["genUp4S_PDG_0_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Hc["genUp4S_PDG_0_0_0_1"].abs() == 211.0)  & # 211=pion
            ((df_Hc["genUp4S_PDG_0_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_0_0_0_2"].isna() == True) | # OR no particle
            (df_Hc["genUp4S_PDG_0_0_0_2"].abs() == 111.0)) & # OR 111=pi0
            ((df_Hc["genUp4S_PDG_0_0_0_3"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_0_0_0_3"].isna() == True)) & # OR no particle
            ((df_Hc["genUp4S_PDG_0_0_0_4"].abs() == 111.0)  | # 111=pi0
            (df_Hc["genUp4S_PDG_0_0_0_4"].isna() == True)) & # OR no particle
            (df_Hc["genUp4S_PDG_0_0_0_5"].isna() == True)) # no particle
              |
        ((df_Hc["Bsig_uniqParID"] == 83886082.0)  &
            (df_Hc["genUp4S_PDG_1_0_0"].abs() == 421.0)  & # 421=D0
            (df_Hc["genUp4S_PDG_1_0_1"].abs() == 211.0)  & # 211=pion
            (df_Hc["genUp4S_PDG_1_0_2"].isna() == True)  & # no particle 
            (df_Hc["genUp4S_PDG_1_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Hc["genUp4S_PDG_1_0_0_1"].abs() == 211.0)  & # 211=pion
            ((df_Hc["genUp4S_PDG_1_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_1_0_0_2"].isna() == True) | # OR no particle
            (df_Hc["genUp4S_PDG_1_0_0_2"].abs() == 111.0)) & # OR 111=pi0
            ((df_Hc["genUp4S_PDG_1_0_0_3"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_1_0_0_3"].isna() == True)) & # OR no particle
            ((df_Hc["genUp4S_PDG_1_0_0_4"].abs() == 111.0)  | # 111=pi0
            (df_Hc["genUp4S_PDG_1_0_0_4"].isna() == True)) & # OR no particle
            (df_Hc["genUp4S_PDG_1_0_0_5"].isna() == True)) # no particle
     ]

In [120]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [121]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

1205717
96244


## merge x,y,z  vertex info of Hc to FSPs

In [140]:
for coordinate in ["x","y","z"]:
    print(df_Hc[coordinate].describe())    
    print(df_Hc[f'{coordinate}_uncertainty'].describe())
    print("\n")
    

count               96244.00
mean                   -0.04
std                     0.39
min                   -16.84
25%                    -0.06
50%                    -0.05
75%                    -0.02
max                    20.43
Name: x, dtype: float64
count               96244.00
mean                    0.05
std                     0.20
min                     0.00
25%                     0.00
50%                     0.00
75%                     0.02
max                    24.46
Name: x_uncertainty, dtype: float64


count               96244.00
mean                    0.02
std                     0.39
min                   -18.44
25%                     0.00
50%                     0.02
75%                     0.03
max                    17.68
Name: y, dtype: float64
count               96244.00
mean                    0.05
std                     0.24
min                     0.00
25%                     0.00
50%                     0.00
75%                     0.02
max            

In [145]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","x","x_uncertainty","y","y_uncertainty","z","z_uncertainty"]], on="__event__")

## save the dfs as they will be used for data prod

In [146]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Hc

In [147]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/'

In [148]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## load the df's

In [6]:
df_FSPs = pd.read_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv".format(subset_size))
df_Y4S_final = pd.read_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## beginning of data prod

In [126]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 1
maxFSPs: 23 

num Events: 96244 



In [149]:
variable_combinations = [
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID","protonID"],"NAHSA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID","protonID"],"NAHSA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID","protonID"],"NAHSA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ,"protonID"],"NAHSA_no_muID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"           ],"NAHSA_no_proID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_allExtras"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_allExtras_xyz"],    
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","x_uncertainty","y","y_uncertainty","z","z_uncertainty"],"NAHSA_allExtras_xyz_uncer"],
    [["M","charge","pionID","kaonID","electronID","muonID"],"NAHSA_IDchargeM"],
    [["M","charge"],"NAHSA_chargeM"],
    [["charge","pionID","kaonID","electronID","muonID"],"NAHSA_chargeIDs"],
    [["charge","pionID"],"NAHSA_chargePionID"],
    #[[],"NAHSA_only4mom"],
    [["charge"],"NAHSA_charge"],
    [["x","x_uncertainty","y","y_uncertainty","z","z_uncertainty"],"NAHSA_Hc_xyz_uncertain"],
    [["x","y","z"],"NAHSA_Hc_xyz"]
    
]

In [127]:
variable_combinations = [
    [[],"NAHSA_only4mom"]
]

In [150]:
minFSPs = 5 # minimum: Bsig: K pi pi lepton, X: 1 

In [ ]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    



                global_tag_Info = str((particle["mcPDG"])) 
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.1

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_Gmodes_1/NAHS_allEvts_oneSub/NAHSA_no_M is True 

numEvents: 1232
num_FSPs_toData: 5
leaves.shape: (1232, 5, 14)
SA_target.shape: (1232, 5)
global_tag.shape: (1232, 6)
leaves[0]: [[ 0.14259353 -0.23874733 -0.42714393  0.50969122  0.          0.
   0.          3.          0.91699219 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.79012734  0.27009329  0.09869234  0.97504294 -1.          0.00682038
   0.00862561  2.          0.98535156  0.00096549  0.15175247  0.84725974
   0.00000072  0.00002158]
 [-0.30713138 -0.09685621 -0.17984667  0.39437993  1.          0.00119517
   0.00537646  2.          0.92382812  0.91642372  0.          0.05465339
   0.02892289  0.        ]
 [-0.32595828  0.89728135  0.53782165  1.09572579 -1.          0.00434325
   0.01232462  2.          0.98046875  0.0000001   0.          0.9999999
   0.          0.        ]
 [ 0.08999581 -0.10610624 -0.10206289  0.1725532


numEvents: 9179
num_FSPs_toData: 11
leaves.shape: (9179, 11, 14)
SA_target.shape: (9179, 11)
global_tag.shape: (9179, 12)
leaves[0]: [[  0.11262705   0.38826191  -0.04237363   0.4297762    1.
    0.00503318   0.01779051   2.           0.828125     0.99999338
    0.           0.           0.00000662   0.        ]
 [  0.11877039  -0.04846106   0.0537724    0.13909116   0.
    0.           0.           2.           0.96484375  -1.
   -1.          -1.          -1.          -1.        ]
 [  0.45995867   0.34882525   0.55719155   0.8143607    1.
    0.02532729   0.0096378    2.           0.94824219   0.65201295
    0.           0.           0.34798705   0.        ]
 [  0.05562339  -0.21132517   0.04163673   0.2626134   -1.
    0.91074547 -19.89052408  -1.          -1.           0.95649555
    0.00000001   0.00000045   0.04350399   0.        ]
 [ -0.0667891    0.00932067  -0.04959819   0.08371164   0.
    0.           0.           2.           0.97460938  -1.
   -1.          -1.          -1.

In [ ]:
print("saving is done")
now = datetime.now()
print("time at end =", now)